In [2]:
import pandas as pd
import os

base_dir = '../data/room'

In [3]:
# mengambil data gamber yang telah di bagi dala base directory
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [4]:
# malakukan image prepocesing dengan keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./225,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./225
)

In [5]:
# menyiapkan data train dan val dari directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=4,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=4,
    class_mode='binary'
)

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [6]:
# melakukan clasifikasi gambar
import keras
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(152, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [7]:
# kompile model dengan adam optimizer loss function
import tensorflow as tf
model.compile(
    loss='binary_crossentropy',
    optimizer= tf.optimizers.Adam(),
    metrics=['accuracy']
)

In [8]:
# sekrang kita untuk melatih model
model.fit(
    train_generator,
    steps_per_epoch=25,
    epochs=20,
    validation_data=val_generator,
    validation_steps=5,
    verbose=2
)

Epoch 1/20
25/25 - 8s - loss: 0.7392 - accuracy: 0.5600 - val_loss: 0.7073 - val_accuracy: 0.5000 - 8s/epoch - 317ms/step
Epoch 2/20
25/25 - 5s - loss: 0.6973 - accuracy: 0.4900 - val_loss: 0.6868 - val_accuracy: 0.8000 - 5s/epoch - 211ms/step
Epoch 3/20
25/25 - 5s - loss: 0.6984 - accuracy: 0.6400 - val_loss: 0.6971 - val_accuracy: 0.5000 - 5s/epoch - 200ms/step
Epoch 4/20
25/25 - 5s - loss: 0.6576 - accuracy: 0.5300 - val_loss: 1.7220 - val_accuracy: 0.5000 - 5s/epoch - 208ms/step
Epoch 5/20
25/25 - 5s - loss: 0.8308 - accuracy: 0.4500 - val_loss: 0.6930 - val_accuracy: 0.5000 - 5s/epoch - 195ms/step
Epoch 6/20
25/25 - 5s - loss: 0.6929 - accuracy: 0.6000 - val_loss: 0.6914 - val_accuracy: 0.7500 - 5s/epoch - 196ms/step
Epoch 7/20
25/25 - 6s - loss: 0.6927 - accuracy: 0.5300 - val_loss: 0.6842 - val_accuracy: 0.6500 - 6s/epoch - 228ms/step
Epoch 8/20
25/25 - 5s - loss: 0.7100 - accuracy: 0.6300 - val_loss: 0.6864 - val_accuracy: 0.6000 - 5s/epoch - 199ms/step
Epoch 9/20
25/25 - 5s - 

In [9]:
!mkdir saved_models

In [10]:
from joblib import dump
dump(model, './saved_models/model.joblib')

['./saved_models/model.joblib']